## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-08-16-54-45 +0000,nyt,New Banksy Work Seen as Comment on Crackdown o...,https://www.nytimes.com/2025/09/08/world/europ...
1,2025-09-08-16-54-23 +0000,nypost,US fast food chain Wendy’s set to make Austral...,https://nypost.com/2025/09/08/lifestyle/us-fas...
2,2025-09-08-16-52-00 +0000,wsj,"After Fatal Stabbing of Ukrainian Refugee, Tru...",https://www.wsj.com/us-news/trump-crime-north-...
3,2025-09-08-16-51-00 +0000,wsj,Supreme Court Lifts Limits on Immigration Raid...,https://www.wsj.com/us-news/law/supreme-court-...
4,2025-09-08-16-46-00 +0000,wsj,Opinion | The Economics of Education Reform,https://www.wsj.com/opinion/the-economics-of-e...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2423/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,28
0,new,12
339,raid,11
61,city,10
27,immigration,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
51,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...,69
76,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...,63
75,2025-09-08-13-52-53 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...,60
86,2025-09-08-13-02-16 +0000,nyt,"Trump Downplays Post Threatening Chicago, Sayi...",https://www.nytimes.com/2025/09/07/us/politics...,58
209,2025-09-07-23-14-55 +0000,bbc,South Korean worker tells BBC of panic during ...,https://www.bbc.com/news/articles/c5yqg0rln74o...,56


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
51,69,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...
183,44,2025-09-08-03-34-58 +0000,nypost,Father on run with 3 kids in wilderness for 4 ...,https://nypost.com/2025/09/07/world-news/fugit...
184,43,2025-09-08-03-32-23 +0000,bbc,What it was like inside court as mushroom murd...,https://www.bbc.com/news/videos/c15k4y5nwz7o?a...
69,40,2025-09-08-14-06-44 +0000,nypost,Police in Nepal open fire on protesters outsid...,https://nypost.com/2025/09/08/world-news/polic...
99,37,2025-09-08-11-46-03 +0000,nypost,Israel threatens to destroy Gaza City in a ‘mi...,https://nypost.com/2025/09/08/world-news/israe...
100,33,2025-09-08-11-38-18 +0000,wapo,"Outrage, confusion in South Korea after Georgi...",https://www.washingtonpost.com/world/2025/09/0...
76,32,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...
129,31,2025-09-08-09-46-24 +0000,nypost,Police see ‘terrorist attack’ in Jerusalem bus...,https://nypost.com/2025/09/08/world-news/four-...
33,29,2025-09-08-15-49-00 +0000,wsj,Payrolls continued to fall across the U.S. las...,https://www.wsj.com/economy/jobs/employment-su...
229,27,2025-09-07-20-56-17 +0000,nypost,Trump goes off on reporter for asking if he’s ...,https://nypost.com/2025/09/07/us-news/trump-go...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
